# Semi-Automated Plate Definition Generation

- tags: #platedefinition #resourcemovement #plateadapter #hamiltonstar
- Last updated: 2026-01-26

## Prerequisites

- Machines used:
  - Hamilton STAR
- Non-PLR dependencies: None 


## Preview of Machine Behvaiour

<video width="640" controls autoplay loop>
  <source src="./assets/star_movement_plate_to_alpaqua_core/animation.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>

## Master Information

i.e. declare parameters that might change from run to run

In [1]:
protocol_mode = "execution" # "execution" or "simulation"

In [2]:
# Choose how many channels you want to be involed in probing actions (1-6 is acceptable)
n_channels = 3

---
## Import Statements

### Non-PLR Dependencies

None

### Machine & Visualizer

In [3]:
%load_ext autoreload
%autoreload 2
       
import random 
import time
import asyncio
    
from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.resources.hamilton import STARLetDeck, STARDeck
from pylabrobot.visualizer.visualizer import Visualizer
from pylabrobot.utils import chunk_list

if protocol_mode == "execution":

  from pylabrobot.liquid_handling.backends import STARBackend

  star = STARBackend()

elif protocol_mode == "simulation":

  from pylabrobot.liquid_handling.backends.hamilton.STAR_chatterbox import STARChatterboxBackend
    
  star = STARChatterboxBackend()

shutdown_executed = False

### Required Resources

In [4]:
from pylabrobot.resources import (
    hamilton_mfx_carrier_L5_base,
    hamilton_mfx_plateholder_DWP_metal_tapped,
    hamilton_mfx_plateholder_DWP_flat,
    Coordinate
)

#### Create Initial Plate Definition

Ideally, the manufactuer publishes technical drawings which act as a starting material that requires verification in physical reality + measurements of missing parameters:

![sd](./assets/star_semiautomated_plate_definition/biorad_96_wellplate_200uL_Vb.png)

```{note}
PyLabRobot always uses "front-left-bottom" references!
This means that every technical drawing measurement has to be converted accordingly:
e.g. `dx = 14.38 (left to center_x) - 5.46 / 2 (radius of well)`.

Furthermore, this is a special plate: its well spacing is symmetric in both the x and y dimensions!
This is not common, and you should always perform a multiple sanity checks to ensure you are not taking measurements from the incorrect direction: e.g. `127.76 - (14.38)*2 = 99.0` mm -> 1st column center_x and 12th column center_x are exactly 99.0 mm apart as expected based on the ANSI/SLAS 4-2004 standard and an 11 column distance -> the plate is symmetric across its welll spacing in the `x` dimension
```

```{warning}
It is now clear whether `16.06-14.81 = 1.25` refers to the `dz` with or without the `material_z_thickness`.
This means without measurement we risk crashing pipette heads into the well cavity bottom - if a plate holder with a pedestal (i.e. central elevation) is used.
```

In [5]:
from pylabrobot.resources.plate import Plate
from pylabrobot.resources.utils import create_ordered_items_2d
from pylabrobot.resources.well import (
    CrossSectionType,
    Well,
    WellBottomType,
)

def biorad_96_wellplate_200uL_Vb(name: str, with_lid: bool = False) -> Plate:
    """Bio-rad cat. no.: HSP9601 (50/package), HSP9601B (8*50/package).

    Bio-rad plate with 96-wells (V-bottoms).
    "The patented rigid 2-component design is specifically engineered to
      withstand the stresses of thermal cycling." -> excellent for automation!

    - Colour: white shell/clear well
    - alternative cat. no.:
      - red shell (HSP-9611)
      - yellow shell (HSP-9621)
      - blue shell (HSP-9631)
      - green shell (HSP-9641)
      - black shell (HSP-9661)
    - Material: Polypropylene
    - Cleanliness: "Certified to be free of DNase, RNase, and human genomic DNA"
    - Total volume/well = 200 uL
    - URL: https://www.bio-rad.com/en-uk/sku/HSP9601-hard-shell-96-well-pcr-plates-
      low-profile-thin-wall-skirted-white-clear?ID=HSP9601
    - technical drawing: ./assets/star_semiautomated_plate_definition/biorad_96_wellplate_200uL_Vb.png
    """
    well_diameter = 5.46  # mm
    return Plate(
        name=name,
        size_x=127.76,
        size_y=85.48,
        size_z=16.06,
        lid=None,
        model=biorad_96_wellplate_200uL_Vb.__name__,
        ordered_items=create_ordered_items_2d(
            Well,
            num_items_x=12,
            num_items_y=8,
            dx=round(14.38-well_diameter/2, 2), # symmetric well spacing across X centerline
            dy=round(11.24-well_diameter/2, 2), # symmetric well spacing across Y centerline
            dz=1.1, # initial guess
            item_dx=9.0,
            item_dy=9.0,
            size_x=well_diameter,
            size_y=well_diameter,
            size_z=14.4,
            bottom_type=WellBottomType.V,
            material_z_thickness=0.65, # initial guess
            cross_section_type=CrossSectionType.CIRCLE,
            compute_volume_from_height=None,
            compute_height_from_volume=None,
        ),
    )

# Modify the above if you want to change/create your own plate definition
# then simply change this variable which is used for the rest of this recipe
plate_definition_function = biorad_96_wellplate_200uL_Vb

In [6]:
14.38-5.46/2

11.65

## Utility Functions

Inevitably we have to create specialised solutions for specialised problems.
Python makes this easy by quickly generating utility functions.
These can be stored with you 'automated Protocol' script or, if used repeadetly be added to your internal repository, or even added to the `pylabrobot.utils` package.

In [7]:
def generate_channel_wells(plate: Plate, channels_used: int):
    """
    Generate a list of wells for multi-channel pipetting with evenly spaced rows.
    
    Returns:
        List of well names:
        e.g., generate_channel_wells(test_plate_0, 4) -> ["A1", "C1", "E1", "H1", "A2", "C2", ...])
    """
    num_rows, num_cols = plate.num_items_y, plate.num_items_x
    
    # Generate evenly spaced row indices
    row_indices = [round(i * (num_rows - 1) / (channels_used - 1)) for i in range(channels_used)]
    
    # Generate well names: all columns, with selected rows per column
    return [f"{chr(ord('A') + row)}{col}" 
            for col in range(1, num_cols + 1) 
            for row in row_indices]

## Instantiate Frontend & Connect to Machine

In [8]:
deck = STARDeck()
lh = LiquidHandler(backend=star, deck=deck)

await lh.setup()

vis = Visualizer(resource=lh)
await vis.setup()

await star.disable_cover_control() # 😈

2026-01-28 18:23:49,585 - pylabrobot.io.usb - INFO - Finding USB device...
2026-01-28 18:23:49,618 - pylabrobot.io.usb - INFO - Found USB device.
2026-01-28 18:23:49,621 - pylabrobot.io.usb - INFO - Found endpoints. 
Write:
       ENDPOINT 0x2: Bulk OUT ===============================
       bLength          :    0x7 (7 bytes)
       bDescriptorType  :    0x5 Endpoint
       bEndpointAddress :    0x2 OUT
       bmAttributes     :    0x2 Bulk
       wMaxPacketSize   :   0x40 (64 bytes)
       bInterval        :    0x0 
Read:
       ENDPOINT 0x81: Bulk IN ===============================
       bLength          :    0x7 (7 bytes)
       bDescriptorType  :    0x5 Endpoint
       bEndpointAddress :   0x81 IN
       bmAttributes     :    0x2 Bulk
       wMaxPacketSize   :   0x40 (64 bytes)
       bInterval        :    0x0


Websocket server started at http://127.0.0.1:2121
File server started at http://127.0.0.1:1337 . Open this URL in your browser.


'C0CDid0016er00/00'

## Configure Deck Layout

In [9]:
# Setup MFX Carrier with DWP PlateHolder that has a pedestal

plateholder_pedestal_0 = hamilton_mfx_plateholder_DWP_metal_tapped(
    name=f"plateholder_pedestal_0"
)

mfx_carrier_0 = hamilton_mfx_carrier_L5_base(
  name="mfx_carrier_0",
  modules={0: plateholder_pedestal_0}
)

deck.assign_child_resource(mfx_carrier_0, rails=10)

# Setup MFX Carrier with DWP PlateHolder that has a flat bottom

plateholder_flat_0 = hamilton_mfx_plateholder_DWP_flat(name=f"plateholder_flat_0")

mfx_carrier_1 = hamilton_mfx_carrier_L5_base(
  name="mfx_carrier_1",
  modules={0: plateholder_flat_0}
)

deck.assign_child_resource(mfx_carrier_1, rails=17)

In [10]:
# Add plate onto PlateHolder with the pedestal

test_plate_0 = plate_definition_function(name="test_plate_0")

mfx_carrier_0[0] = test_plate_0 

Opening in existing browser session.


### Step Calculations

In [11]:
wells_to_probe = generate_channel_wells(test_plate_0, channels_used=n_channels)

if len(wells_to_probe) > 10: # You might want to use this recipe to create a 6-wellplate definition
    print(wells_to_probe[:10])

['A1', 'E1', 'H1', 'A2', 'E2', 'H2', 'A3', 'E3', 'H3', 'A4']


[289359:289359:0100/000000.565807:ERROR:content/zygote/zygote_linux.cc:673] write: Broken pipe (32)


---
## Execution

In [12]:
recipe_execution_start_time = time.time()

### Pickup Teaching Needles & CORE Grippers

In [13]:
teaching_tip_rack = lh.deck.get_resource("teaching_tip_rack")
await lh.pick_up_tips(teaching_tip_rack["A1:C1"], use_channels=[0,1,2,3,4,5][:n_channels])

await star.pick_up_core_gripper_tools(front_channel=7)

'C0ZTid0019er00/00sx000 000 000 000 000 000 000 000sg000 000 000 000 000 000 486 509'

### 1) Focus: Flat PlateHolder -> `dz` + `material_z_thickness`

#### 1.a.) Probe True PlateHolder Origin

In [39]:
plate_holder_pedestal_child_coord = mfx_carrier_0[0].get_absolute_location()+mfx_carrier_0[0].child_location

plate_holder_flat_child_coord = mfx_carrier_1[0].get_absolute_location()+mfx_carrier_1[0].child_location

test_location_w_offset = plate_holder_flat_child_coord + Coordinate(x=3, y=3, z=0)

In [15]:
from tqdm.auto import tqdm

In [16]:
# You might have deviations in positional accuracy across your 1000uL channels
# (this is what the service/Hamilton-executed calibration is mitigating)
# To capture this information, perform measurements with multiple channels!

front_pos_master_list, left_pos_master_list = [], []
plate_holder_flat_top_surface = []

for channel_idx in tqdm(range(n_channels)):
    
    # Prepare Z position of X-Y probing
    await star.move_all_channels_in_z_safety()
    await star.prepare_for_manual_channel_operation(channel=channel_idx)
    
    await star.move_channel_x(x=test_location_w_offset.x, channel=channel_idx)
    await star.move_channel_y(y= test_location_w_offset.y, channel=channel_idx)
    
    probed_z_position = await star.clld_probe_z_height_using_channel(
            channel_idx=channel_idx,
            channel_speed=6.0, start_pos_search=test_location_w_offset.z+20,
            detection_edge=5, post_detection_dist=2.0,
            move_channels_to_safe_pos_after=False
        ) if protocol_mode == "execution" else plate_holder_flat_child_coord.z
    plate_holder_flat_top_surface.append(probed_z_position)
    
    await star.move_channel_z(z=probed_z_position+1.0, channel=channel_idx)
    
    # X-Y Probing
    offset_n_replicates = 3
    
    front_pos_list = []
    for x in range(offset_n_replicates):
        probed_front_position = await star.clld_probe_y_position_using_channel(
            channel_idx=channel_idx,
            probing_direction="forward",
            channel_speed=3,
            post_detection_dist=3.0,
            end_pos_search=test_location_w_offset.y-5
        ) if protocol_mode == "execution" else plate_holder_flat_child_coord.y
        front_pos_list.append(probed_front_position)
    
    left_pos_list =[]
    for x in range(offset_n_replicates):
        probed_left_position = await star.clld_probe_x_position_using_channel(
            channel_idx=channel_idx,
            probing_direction="left",
            post_detection_dist=1.0,
            end_pos_search=test_location_w_offset.x-5
        ) if protocol_mode == "execution" else plate_holder_flat_child_coord.x
        left_pos_list.append(probed_left_position)
    
    await star.move_all_channels_in_z_safety()

    print(f"{channel_idx}: {left_pos_list=} | {front_pos_list=}")

    front_pos_master_list.extend(front_pos_list)
    left_pos_master_list.extend(left_pos_list)
    
true_plate_holder_flat_child_coord = Coordinate(
    x=round(sum(left_pos_master_list)/len(left_pos_master_list), 1),
    y=round(sum(front_pos_master_list)/len(front_pos_master_list), 1),
    z=probed_z_position,
)

offset = true_plate_holder_flat_child_coord - plate_holder_flat_child_coord
print(
    f"\nOffset of {offset} between plate_holder child location modelled "
    "and physical reality detected!"
)

  0%|          | 0/3 [00:00<?, ?it/s]

0: left_pos_list=[463.1, 463.1, 463.1] | front_pos_list=[71.7, 71.9, 69.2]
1: left_pos_list=[463.0, 463.0, 463.0] | front_pos_list=[72.0, 72.0, 72.0]
2: left_pos_list=[463.2, 463.2, 463.2] | front_pos_list=[71.6, 71.7, 71.7]

Offset of Coordinate(-00.900, 000.000, 001.400) between plate_holder child location modelled and physical reality detected!


#### 1.b) Move Plate to Flat PlateHolder

In [17]:
move_target = mfx_carrier_1[0]

if protocol_mode == "simulation":
  time.sleep(2)
    
await lh.move_plate(
  plate=test_plate_0,
  to=move_target,
  use_arm="core",
  pickup_distance_from_top=6,
  core_grip_strength=40,
  return_core_gripper=False,
)

if protocol_mode == "execution":
  # "smart" command, will ask operator for input if it cannot find plate in
  # move_target location place into condition for simulation mode

  # (1) check transfer success, (2) push plate flush
  await star.core_check_resource_exists_at_location_center(
    location=test_plate_0.get_absolute_location(),
    resource=test_plate_0,
    gripper_y_margin=9,
    enable_recovery=True,
    audio_feedback=False,
  )

print(star.core_parked)

if protocol_mode == "simulation":
  time.sleep(2)

False


#### 1.c) Plate Height Probing

To probe the height of the plate we do not want to probe the center because it might bend under the probing force.
Instead use some of the sturdy edges:

In [18]:
channel_idx = 0

well_A1 = test_plate_0["A1"][0]
plate_height_test_coord = well_A1.get_absolute_location(
    "left", "back", "top"
) + offset + Coordinate(-2, 2, 0.0)

await lh.backend.move_all_channels_in_z_safety()

await lh.backend.prepare_for_manual_channel_operation(channel=channel_idx)

await lh.backend.move_channel_x(x=plate_height_test_coord.x, channel=channel_idx)
await lh.backend.move_channel_y(y=plate_height_test_coord.y, channel=channel_idx)

probed_plate_top_results = []
probed_plate_top = await star.ztouch_probe_z_height_using_channel(
    channel_idx=channel_idx,
    channel_speed=5.0,
    start_pos_search=250, # starting point above 245 safety plane
    move_channels_to_safe_pos_after=False  # must be False to avoid STAR C0 module
)
probed_plate_top_results.append(probed_plate_top)

for idx in range(2):
    replicate_probed_plate_top = await star.ztouch_probe_z_height_using_channel(
        channel_idx=channel_idx,
        channel_speed=5.0,
        start_pos_search=probed_plate_top+2.0,
        move_channels_to_safe_pos_after=False  # must be False to avoid STAR C0 module
    )
    probed_plate_top_results.append(replicate_probed_plate_top)

await lh.backend.move_all_channels_in_z_safety()


mean_probed_plate_top = sum(probed_plate_top_results)/len(probed_plate_top_results)
mean_plate_holder_flat_top_surface = sum(plate_holder_flat_top_surface)/len(plate_holder_flat_top_surface)

measured_plate_size_z = round(mean_probed_plate_top - mean_plate_holder_flat_top_surface, 2)

print(
    f"Currently modelled plate height = {test_plate_0}\n"
    f"Measured plate height = {measured_plate_size_z}"
)

Currently modelled plate height = Plate(name='test_plate_0', size_x=127.76, size_y=85.48, size_z=16.06, location=Coordinate(004.000, 003.500, 059.805))
Measured plate height = 14.84


In [19]:
plate_holder_flat_top_surface

[179.5, 179.4, 179.4]

In [20]:
probed_plate_top

194.27

#### 1.d) Interactive X-Y Verification of Bottom-Left Well

In [21]:
channel_idx=0

In [22]:
test_location = test_plate_0.column(0)[-1].get_absolute_location(
    x="center",y="center",z="top"
) + offset

await lh.backend.move_all_channels_in_z_safety()

await lh.backend.prepare_for_manual_channel_operation(channel=channel_idx)

Position channel with teaching needle above bottom-left well

In [23]:
# Manually modify `x_offset` if teaching needle is not in the center of the well
x_offset = 0.0
test_x = test_location.x + x_offset

await lh.backend.move_channel_x(x=test_x, channel=channel_idx)

current_dx = round(
    test_x - true_plate_holder_flat_child_coord.x - test_plate_0.children[0].get_size_x()/2,
    2)
print(f"{test_x}, {current_dx=}")

477.48, current_dx=11.65


Reminder from the technical drawing above: 14.38-5.46/2 = 11.65 mm

In [24]:
# Manually modify `y_offset` if teaching needle is not in the center of the well
y_offset = 0.0
test_y = test_location.y + y_offset

await lh.backend.move_channel_y(y=test_y, channel=channel_idx)

current_dy = round(
    test_y - true_plate_holder_flat_child_coord.y - test_plate_0.children[0].get_size_y()/2,
    2)

print(f"{test_y}, {current_dy=}")

82.74, current_dy=8.51


Reminder from the technical drawing above: 11.24-5.46/2 = 8.51 mm

In [25]:
# Only to move the teaching needle into the well for X/Y correction
test_z = test_location.z + 2.0
await lh.backend.move_channel_z(z=test_z, channel=channel_idx)
test_z

196.9

```{info}
At this point, take note of the `current_dx` and `current_dy`.
These are the actual values you will want to use and update with
```

In [26]:
final_query = input(f"Have you updated the dx and dy?")

await lh.backend.move_all_channels_in_z_safety()

Have you updated the dx and dy? 


'C0ZAid0214er00/00'

#### 1.e) Store Coordinates to Probe on Flat

In [27]:
positions_to_probe_flat = [
    well.get_absolute_location("c", "c", "top") + offset + Coordinate(x_offset, y_offset, 0.0)
    for well in test_plate_0.children
        if well.get_identifier() in wells_to_probe
]

#### 1.f) Z Probing of Wells (on Flat PlateHolder)

In [32]:
if protocol_mode == "execution":

    probing_start_time = time.time()

    await lh.backend.move_all_channels_in_z_safety()
    use_channels = list(range(n_channels))
    
    plate_on_flat_results = []
    
    for positions in tqdm(chunk_list(positions_to_probe_flat, chunk_size=n_channels)):
        
        x_pos = [coord.x for coord in positions]
        y_pos = [coord.y for coord in positions]
        z_start_pos = mean_plate_holder_flat_top_surface + measured_plate_size_z + 5
        
        
        await star.move_channel_x(0, x_pos[0])

        await star.position_channels_in_y_direction(
          {channel: y for channel, y in zip(use_channels, y_pos)}
        )
        await star.position_channels_in_z_direction(
          {channel: z_start_pos for channel in use_channels}
        )

        # Detect well cavity bottoms
        start_pos_search_list = [z_start_pos for x in range(n_channels)]
        measured_well_bottoms_replicate_summary = []
        tech_n_replicates = 3 
        search_speeds = [10.5, 7.0, 8.5, 10.0, 8.8, 9.2, 7.8, 10.3][:n_channels]

        for n in range(tech_n_replicates):

            # Parallelise ztouch probing
            # taking care to avoid parameters which require the unparallelizable C0 command module
            measured_well_bottoms = await asyncio.gather(
                  *[
                      star.ztouch_probe_z_height_using_channel(
                          channel_idx=channel_idx,
                          tip_len=lh.head[channel_idx].get_tip().total_tip_length,  # must be declared to avoid STAR C0 module
                          channel_speed=search_speed,
                          start_pos_search=start_pos_z,
                          lowest_immers_pos=probed_z_position-2,
                          detection_limiter_in_PWM=0,
                          push_down_force_in_PWM=0,
                          post_detection_dist=0.0, # must be 0 to avoid STAR C0 module
                          move_channels_to_safe_pos_after=False  # must be False to avoid STAR C0 module
                    )
                    for channel_idx, start_pos_z, search_speed in zip(
                        use_channels, start_pos_search_list, search_speeds
                    )
                  ]
                )
            measured_well_bottoms_replicate_summary.append(measured_well_bottoms)

            start_pos_search_list = [x+5 for x in measured_well_bottoms] # Accelerate

        mean_measured_well_bottoms = [
            round(sum(replicates)/tech_n_replicates, 2)
            for replicates in list(zip(*measured_well_bottoms_replicate_summary))
        ]
        plate_on_flat_results.extend(mean_measured_well_bottoms)
        
        # SAFETY: move UP before starting next loop!
        await star.position_channels_in_z_direction(
          {channel: z_start_pos for channel in use_channels}
        )

    await lh.backend.move_all_channels_in_z_safety()
    
    print(f"Well probing took {time.time() - probing_start_time} sec")
    

  0%|          | 0/12 [00:00<?, ?it/s]

Well probing took 107.66316056251526 sec


In [33]:
min(plate_on_flat_results), max(plate_on_flat_results)

(180.45, 180.86)

#### 1.f) Move Plate back onto tapped PlateHolder

In [34]:
move_target = mfx_carrier_0[0]

await lh.move_plate(
  plate=test_plate_0,
  to=move_target,
  use_arm="core",
  pickup_distance_from_top=6,
  core_grip_strength=40,
  return_core_gripper=False,
)

if protocol_mode == "execution":

  await star.core_check_resource_exists_at_location_center(
    location=test_plate_0.get_absolute_location(),
    resource=test_plate_0,
    gripper_y_margin=9,
    enable_recovery=True,
    audio_feedback=False,
  )

#### 1.g) Z Probing of Flat PlateHolder

In [35]:
if protocol_mode == "execution":

    probing_start_time = time.time()

    await lh.backend.move_all_channels_in_z_safety()
    use_channels = list(range(n_channels))
    
    plate_holder_FLAT_results = []
    
    for positions in tqdm(chunk_list(positions_to_probe_flat, chunk_size=n_channels)):
        
        x_pos = [coord.x for coord in positions]
        y_pos = [coord.y for coord in positions]
        z_start_pos =  mean_plate_holder_flat_top_surface + 5
        
        await star.move_channel_x(0, x_pos[0])

        await star.position_channels_in_y_direction(
          {channel: y for channel, y in zip(use_channels, y_pos)}
        )
        await star.position_channels_in_z_direction(
          {channel: z_start_pos for channel in use_channels}
        )

        # Detect well cavity bottoms
        start_pos_search_list = [z_start_pos for x in range(n_channels)]
        measured_well_bottoms_replicate_summary = []
        tech_n_replicates = 3 
        search_speeds = [10.5, 7.0, 8.5, 10.0, 8.8, 9.2, 7.8, 10.3][:n_channels]

        for n in range(tech_n_replicates):

            # Parallelise ztouch probing
            # taking care to avoid parameters which require the unparallelizable C0 command module
            measured_well_bottoms = await asyncio.gather(
                  *[
                      star.ztouch_probe_z_height_using_channel(
                          channel_idx=channel_idx,
                          tip_len=lh.head[channel_idx].get_tip().total_tip_length,  # must be declared to avoid STAR C0 module
                          channel_speed=search_speed,
                          start_pos_search=start_pos_z,
                          lowest_immers_pos=probed_z_position-2,
                          detection_limiter_in_PWM=0,
                          push_down_force_in_PWM=0,
                          post_detection_dist=0.0, # must be 0 to avoid STAR C0 module
                          move_channels_to_safe_pos_after=False  # must be False to avoid STAR C0 module
                    )
                    for channel_idx, start_pos_z, search_speed in zip(
                        use_channels, start_pos_search_list, search_speeds
                    )
                  ]
                )
            measured_well_bottoms_replicate_summary.append(measured_well_bottoms)

            start_pos_search_list = [x+5 for x in measured_well_bottoms] # Accelerate

        mean_measured_well_bottoms = [
            round(sum(replicates)/tech_n_replicates, 2)
            for replicates in list(zip(*measured_well_bottoms_replicate_summary))
        ]
        plate_holder_FLAT_results.extend(mean_measured_well_bottoms)
        
        # SAFETY: move UP before starting next loop!
        await star.position_channels_in_z_direction(
          {channel: z_start_pos for channel in use_channels}
        )

    await lh.backend.move_all_channels_in_z_safety()
    
    print(f"Well probing took {time.time() - probing_start_time} sec")
    

  0%|          | 0/12 [00:00<?, ?it/s]

Well probing took 83.35548710823059 sec


#### 1.h) Calculate `dz` + `material_z_thickness`

In [53]:
dz_plus_material_z_thickness_results = [
    round(well_bottom-flat_surface_below_it,2)
    for well_bottom,flat_surface_below_it 
    in list(zip(plate_on_flat_results, plate_holder_FLAT_results))
]

chunk_list(dz_plus_material_z_thickness_results, n_channels)

[[1.87, 1.9, 1.79],
 [1.99, 1.93, 1.88],
 [2.07, 2.05, 1.96],
 [2.18, 2.06, 2.08],
 [2.21, 2.18, 2.08],
 [2.33, 2.16, 2.2],
 [2.31, 2.11, 2.23],
 [2.21, 2.17, 2.17],
 [2.2, 2.09, 2.19],
 [2.09, 2.12, 2.09],
 [1.99, 2.02, 1.91],
 [1.88, 1.92, 1.79]]

### 2.) Focus: PlateHolder w/ Pedestal -> `material_z_thickness`

#### 2.a) Store Coordinates to Probe on Pedestal

In [45]:
positions_to_probe_pedestal = [
    well.get_absolute_location("c", "c", "top") + Coordinate(x_offset, y_offset, 0)
    for well in test_plate_0.children
        if well.get_identifier() in wells_to_probe
]

#### 2.b) Z Probing of Wells (on Pedestal)

In [46]:
if protocol_mode == "execution":

    probing_start_time = time.time()

    await lh.backend.move_all_channels_in_z_safety()
    use_channels = list(range(n_channels))
    
    plate_on_PEDESTAL_results = []
    
    for positions in tqdm(chunk_list(positions_to_probe_pedestal, chunk_size=n_channels)):
        
        x_pos = [coord.x for coord in positions]
        y_pos = [coord.y for coord in positions]
        z_start_pos =  plate_holder_pedestal_child_coord.z + measured_plate_size_z + 5
        
        await star.move_channel_x(0, x_pos[0])

        await star.position_channels_in_y_direction(
          {channel: y for channel, y in zip(use_channels, y_pos)}
        )
        await star.position_channels_in_z_direction(
          {channel: z_start_pos for channel in use_channels}
        )

        # Detect well cavity bottoms
        start_pos_search_list = [z_start_pos for x in range(n_channels)]
        measured_well_bottoms_replicate_summary = []
        tech_n_replicates = 3 
        search_speeds = [10.5, 7.0, 8.5, 10.0, 8.8, 9.2, 7.8, 10.3][:n_channels]

        for n in range(tech_n_replicates):

            # Parallelise ztouch probing
            # taking care to avoid parameters which require the unparallelizable C0 command module
            measured_well_bottoms = await asyncio.gather(
                  *[
                      star.ztouch_probe_z_height_using_channel(
                          channel_idx=channel_idx,
                          tip_len=lh.head[channel_idx].get_tip().total_tip_length,  # must be declared to avoid STAR C0 module
                          channel_speed=search_speed,
                          start_pos_search=start_pos_z,
                          lowest_immers_pos=probed_z_position-2,
                          detection_limiter_in_PWM=0,
                          push_down_force_in_PWM=0,
                          post_detection_dist=0.0, # must be 0 to avoid STAR C0 module
                          move_channels_to_safe_pos_after=False  # must be False to avoid STAR C0 module
                    )
                    for channel_idx, start_pos_z, search_speed in zip(
                        use_channels, start_pos_search_list, search_speeds
                    )
                  ]
                )
            measured_well_bottoms_replicate_summary.append(measured_well_bottoms)

            start_pos_search_list = [x+5 for x in measured_well_bottoms] # Accelerate

        mean_measured_well_bottoms = [
            round(sum(replicates)/tech_n_replicates, 2)
            for replicates in list(zip(*measured_well_bottoms_replicate_summary))
        ]
        plate_on_PEDESTAL_results.extend(mean_measured_well_bottoms)
        
        # SAFETY: move UP before starting next loop!
        await star.position_channels_in_z_direction(
          {channel: z_start_pos for channel in use_channels}
        )

    await star.move_all_channels_in_z_safety()
    
    print(f"Well probing took {time.time() - probing_start_time} sec")
    

  0%|          | 0/12 [00:00<?, ?it/s]

Well probing took 113.91992855072021 sec


#### 2.c) Move Plate to Flat PlateHolder

In [47]:
move_target = mfx_carrier_1[0]

if protocol_mode == "simulation":
  time.sleep(2)
    
await lh.move_plate(
  plate=test_plate_0,
  to=move_target,
  use_arm="core",
  pickup_distance_from_top=6,
  core_grip_strength=40,
  return_core_gripper=False,
)

if protocol_mode == "execution":
  # "smart" command, will ask operator for input if it cannot find plate in
  # move_target location place into condition for simulation mode

  # (1) check transfer success, (2) push plate flush
  await star.core_check_resource_exists_at_location_center(
    location=test_plate_0.get_absolute_location(),
    resource=test_plate_0,
    gripper_y_margin=9,
    enable_recovery=True,
    audio_feedback=False,
  )

print(star.core_parked)

if protocol_mode == "simulation":
  time.sleep(2)

False


#### 2.d) Z Probing of PlateHolder w/ Pedestal

In [48]:
if protocol_mode == "execution":

    probing_start_time = time.time()

    await lh.backend.move_all_channels_in_z_safety()
    use_channels = list(range(n_channels))
    
    plate_holder_PEDESTAL_results = []
    
    for positions in tqdm(chunk_list(positions_to_probe_pedestal, chunk_size=n_channels)):
        
        x_pos = [coord.x for coord in positions]
        y_pos = [coord.y for coord in positions]
        z_start_pos =  plate_holder_pedestal_child_coord.z + 5
        
        await star.move_channel_x(0, x_pos[0])

        await star.position_channels_in_y_direction(
          {channel: y for channel, y in zip(use_channels, y_pos)}
        )
        await star.position_channels_in_z_direction(
          {channel: z_start_pos for channel in use_channels}
        )

        # Detect well cavity bottoms
        start_pos_search_list = [z_start_pos for x in range(n_channels)]
        measured_well_bottoms_replicate_summary = []
        tech_n_replicates = 3 
        search_speeds = [10.5, 7.0, 8.5, 10.0, 8.8, 9.2, 7.8, 10.3][:n_channels]

        for n in range(tech_n_replicates):

            # Parallelise ztouch probing
            # taking care to avoid parameters which require the unparallelizable C0 command module
            measured_well_bottoms = await asyncio.gather(
                  *[
                      star.ztouch_probe_z_height_using_channel(
                          channel_idx=channel_idx,
                          tip_len=lh.head[channel_idx].get_tip().total_tip_length,  # must be declared to avoid STAR C0 module
                          channel_speed=search_speed,
                          start_pos_search=start_pos_z,
                          lowest_immers_pos=probed_z_position-2,
                          detection_limiter_in_PWM=0,
                          push_down_force_in_PWM=0,
                          post_detection_dist=0.0, # must be 0 to avoid STAR C0 module
                          move_channels_to_safe_pos_after=False  # must be False to avoid STAR C0 module
                    )
                    for channel_idx, start_pos_z, search_speed in zip(
                        use_channels, start_pos_search_list, search_speeds
                    )
                  ]
                )
            measured_well_bottoms_replicate_summary.append(measured_well_bottoms)

            start_pos_search_list = [x+5 for x in measured_well_bottoms] # Accelerate

        mean_measured_well_bottoms = [
            round(sum(replicates)/tech_n_replicates, 2)
            for replicates in list(zip(*measured_well_bottoms_replicate_summary))
        ]
        plate_holder_PEDESTAL_results.extend(mean_measured_well_bottoms)
        
        # SAFETY: move UP before starting next loop!
        await star.position_channels_in_z_direction(
          {channel: z_start_pos for channel in use_channels}
        )

    await star.move_all_channels_in_z_safety()
    
    print(f"Well probing took {time.time() - probing_start_time} sec")
    

  0%|          | 0/12 [00:00<?, ?it/s]

Well probing took 83.25026893615723 sec


#### 2.e) Calculate `material_z_thickness`

In [49]:
material_z_thickness_results = [
    round(well_bottom-flat_surface_below_it,2)
    for well_bottom,flat_surface_below_it 
    in list(zip(plate_on_PEDESTAL_results, plate_holder_PEDESTAL_results))
]

chunk_list(material_z_thickness_results, n_channels)

[[0.65, 0.71, 0.68],
 [0.68, 0.74, 0.7],
 [0.76, 0.8, 0.75],
 [0.82, 0.87, 0.83],
 [0.87, 0.94, 0.83],
 [0.89, 0.89, 0.87],
 [0.88, 0.88, 0.89],
 [0.84, 0.84, 0.9],
 [0.82, 0.91, 0.88],
 [0.79, 0.85, 0.78],
 [0.71, 0.83, 0.71],
 [0.65, 0.78, 0.68]]

### 3.) Update Summary for Plate Definition

In [59]:
dz_results = [
    round(combined-m_thickness,2)
    for combined, m_thickness
    in list(zip(dz_plus_material_z_thickness_results, material_z_thickness_results))
]

chunk_list(dz_results, n_channels)

[[1.22, 1.19, 1.11],
 [1.31, 1.19, 1.18],
 [1.31, 1.25, 1.21],
 [1.36, 1.19, 1.25],
 [1.34, 1.24, 1.25],
 [1.44, 1.27, 1.33],
 [1.43, 1.23, 1.34],
 [1.37, 1.33, 1.27],
 [1.38, 1.18, 1.31],
 [1.3, 1.27, 1.31],
 [1.28, 1.19, 1.2],
 [1.23, 1.14, 1.11]]

#### `dz` & `material_z_thickness`

In [61]:
mean_material_z_thickness = round(
    sum(material_z_thickness_results)/len(material_z_thickness_results),
    2
)
mean_dz = round(
    sum(dz_results)/len(dz_results),
    2
)

print(f"{mean_dz=} | {mean_material_z_thickness=}")

mean_dz=1.26 | mean_material_z_thickness=0.8


Reminder from the technical drawing above: 16.06 - 14.82 = 1.24 mm

This shows that the `dz` value was reported in the technical drawing but the `material_z_thickness` was omitted!

#### `dx` & `dy`

In [64]:
print(f"Required pdates to dx = {x_offset}, and dy = {y_offset}")

Updates to dx = 0.0, and dy = 0.0


#### Plate size_z / height

In [66]:
measured_plate_size_z, test_plate_0.get_size_z()

(14.84, 16.06)

If there is a deviation in the plate height, check that whether the wells protrude above the plate.
This is a common plate feature that facilitates heat sealing.

Technical drawings often only report the combined plate and well size_z.

### Return Tools

In [69]:
await lh.backend.return_core_gripper_tools()

channel_tip_occupancy = await lh.backend.request_tip_presence()
if any(channel_tip_occupancy):
    await lh.return_tips()

### Time Report

In [51]:
recipe_execution_end_time = time.time()-recipe_execution_start_time

print(f"Execution of this recipe took {round(recipe_execution_end_time/60, 2)} min")

Execution of this recipe took 19.67 min


## Shutdown

In [70]:
if protocol_mode == 'execution':
    await star.move_all_channels_in_z_safety()

    if star.core96_head_installed:
        await star.head96_move_to_z_safety()

    if star.iswap_installed:
        await star.park_iswap()
    
    if not star.core_parked: # return grippers
        await star.return_core_gripper_tools()
        await star.move_all_channels_in_z_safety()

    # discard tips if any are present
    has_tip_check = [lh.head[idx].has_tip for idx in range(8)]
    if any(has_tip_check):
        await lh.discard_tips()
    else:
        await star.move_channel_x(x=400, channel=0)
    await star.spread_pip_channels()

await lh.stop()

shutdown_executed = True

2026-01-28 19:00:21,016 - pylabrobot.io.usb - WARNING - Closing connection to USB device.


## (Recovery Options)

Useful if you want to quickly remove the teaching needles and/or grippers.

#### Channels

##### Return Tips

In [71]:
if not shutdown_executed:
    channel_tip_occupancy = await lh.backend.request_tip_presence()
    if any(channel_tip_occupancy):
        await lh.return_tips()

---
### Return CORE Grippers

In [72]:
if not shutdown_executed:
    await lh.backend.return_core_gripper_tools()